In [33]:
# Import the 3 datasets for SQuAD 
import json

# Opening JSON 
loc = r'.\Stanford Question Answering Dataset\train-v1.1.json'
with open(loc) as f:
    train = json.load(f)

loc2 = r'.\Stanford Question Answering Dataset\dev-v1.1.json'
with open(loc2) as f:
    dev = json.load(f)

In [38]:
# Stanford Question Answering Dataset (SQuAD)
train = pd.read_csv(r"C:\Users\Kathy\develop\UCSD Bootcamp\Capstone Prep\Stanford Question Answering Dataset (SQuAD)\train.csv")
validation = pd.read_csv(r"C:\Users\Kathy\develop\UCSD Bootcamp\Capstone Prep\Stanford Question Answering Dataset (SQuAD)\validation.csv")

In [ ]:
# Question-Answer Dataset
# qa1 = pd.read_fwf(r"C:\Users\Kathy\develop\UCSD Bootcamp\Capstone Prep\Question-Answer Dataset\S08_question_answer_pairs.txt", delimiter = "\t")
# qa2 = pd.read_fwf(r"C:\Users\Kathy\develop\UCSD Bootcamp\Capstone Prep\Question-Answer Dataset\S09_question_answer_pairs.txt")
# qa3 = pd.read_fwf(r"C:\Users\Kathy\develop\UCSD Bootcamp\Capstone Prep\Question-Answer Dataset\S10_question_answer_pairs.txt")

In [ ]:
# Realize SQuAD is also in datasets package
import pandas as pd
import json
import os
from transformers import pipeline
# https://stackoverflow.com/questions/74586892/no-module-named-keras-saving-hdf5-format
# https://huggingface.co/learn/nlp-course/chapter2/6?fw=pt

print(os.getcwd())

In [1]:
from datasets import load_dataset
squad = load_dataset("squad")

In [4]:
from transformers import pipeline, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
tokenized_squad = squad.map(preprocess_function, batched=True, 
                            remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [9]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# dataset is tokenized and ready to use for model